In [1]:
import numpy as np # importing numpy for matrix operations 
from scipy import *
from scipy.optimize import minimize
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython import display 
import itertools
from IPython.display import display
import random
import operator
from random import choices, randint, randrange, random, sample, seed, uniform, choice
from collections import namedtuple
from typing import List, Optional, Callable, Tuple
from functools import partial
sns.set()
import random
from itertools import product
import re
from collections import OrderedDict
from operator import getitem
from pprint import pprint
from numpy import asarray
import copy
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.colors import ListedColormap
import sys

In [2]:
import warnings
warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning) 

In [3]:
class IterRegistry(type):
    def __iter__(cls):
        return iter(cls._registry)

In [4]:
class MarketConfig: 
    #class attributes
    t0 = 2018 #The starting year for evaluation of the pay-offs
    T = 25  # Planning Horizon t = 2016,...,2040 # 100 year scope 
    year = np.arange(2018, 2018 + T)
    distance = np.array([23000]) #nautical miles #Doudnikoff & Lacoste(2014). 
    port_time =  np.array([240]) #hours ==> 10 days in a year  #Doudnikoff & Lacoste(2014).
    demand_18_21 = np.array([7000000,7200000,7200000,7800000]) #2018-2021 TEU data for europe asia based on unctad 2020in TEU 
    annual_working_time = np.array([6480])  #hours per year, assumption   #hours per year, assumption 
    initial_freight_rate = np.array([822])  ## in US$/TEU 
    #SFOC_main = np.array([206])  #g/kWh, specific daily main engine fuel oil consumption rate
    #SFOC_aux = np.array([221]) # specific fuel oil consumption of the auxiliary engine [g/kW h], 
    eng_load_main = np.array([0.8]) # % 
    eng_load_aux = np.array([0.5])  # engine load of the auxiliary engine [\%]


    def __init__(self, number_of_firms, pollution_decay_parameter, pollution_damage_parameter,
                 demand_income_elasticity, demand_price_elasticity, 
                 freight_rate, fuel_data, discount_rate ):
        '''__init__ a method to describe the poperty that the shipping market and simulation game has'''
        self.number_of_firms = number_of_firms
        self.pollution_decay_parameter = pollution_decay_parameter
        self.pollution_damage_parameter = pollution_damage_parameter
        self.demand_income_elasticity = demand_income_elasticity #Constant income elasticity #IMF
        self.demand_price_elasticity =  demand_price_elasticity #Constant own price elasticity#IMF
        self.freight_rate = freight_rate ## in US$/TEU
        self.fuel_data = fuel_data 
        self.discount_rate = discount_rate 
        #self.bau_industry_emission =  bau_industry_emission 
        #self.bau_pollution_stock = bau_pollution_stock
        #self.market_capacity = 0 #don 't need anymore
         
    
    #Don t need anymore
    #def get_market_capacity (self, firm_instance):
        #self.market_capacity += np.multiply(firm_instance.max_number_of_vessel, firm_instance.capacity) 
        #return self.market_capacity 
    
    def get_freight_rate_ratio(self):
        self.beta = np.true_divide(self.freight_rate, MarketConfig.initial_freight_rate) 
        #should be equal to 1, since we re keeping freight rate cst 
        return self.beta
   
    def get_market_demand(self):
        '''compute market level demand'''
        #---------> 1.Import real GDP growth data & compute GDP ratio : Source IMF@2020 #
        G_df = pd.read_csv('real_growth_rate.csv') #import IMF data
        
        #-------> 2.Construct GDP projection path 2022-2043 based on projection growth data 
        gdp_growth= G_df.values[:,7:28] #Read gdp growth from 2022-2,043; file begins with 2016 growth rates
        g =1 +(gdp_growth/100)  
        #self.gdp = np.array([100.00 for j in range(MarketConfig.T)])
        #for foo in range (1,MarketConfig.T):
        #    self.gdp[foo] = g[:,foo]* self.gdp[foo-1]
            
        #print(self.gdp)
        #--------->  "compute GDP ratio based on IMF@2020"
        gdp_ratio = np.array([np.float64(g[:,0]) for j in range(np.size(g))])
        for moo in range (np.size(g)):
            sub_g = g[:,:moo+1]
            #print(sub_g)
            gdp_ratio[moo] = np.prod(sub_g)  
    
        
        #--------->  "compute freight rate ratio"
        self.freight_rate_ratio_multiplied = np.power(self.beta, self.demand_price_elasticity)
        #print(freight_rate_ratio_multiplied)
        
        #---------> "Project Transport Demand (industry demand)"
        loo =np.multiply(np.power(gdp_ratio, self.demand_income_elasticity), self.freight_rate_ratio_multiplied ) #Will need to be updated in case fuel prices are varied over time  
        predict_demand= np.multiply(loo, MarketConfig.demand_18_21[3])
        self.planing_market_demand = np.concatenate((MarketConfig.demand_18_21, predict_demand), axis=None) #concatenate the predicted values with the Untac data        
        self.market_demand = self.planing_market_demand[:MarketConfig.T]
        #print(Y) # size = 1 dimesion array with size T=35 years
        #self.market_psy = np.multiply(np.power(gdp_ratio, self.demand_income_elasticity), MarketConfig.demand_18_21[3] )
        return self.market_demand #, self.gdp #, self.market_psy

In [5]:
class Firm:
    #class attributes intialisation, to be updated with the addition of each instance
    _registry = []
    
    def __init__(self, index, capacity, design_speed, 
                 min_speed, max_speed, main_engine_power, aux_engine_power,
                 fix_cost, fuel_type, SFOC_M, SFOC_A, market_share):
        '''__init__ a method to describe the poperty that all shipping firm have'''
        self._registry.append(self)
        self.index = index
        self.capacity = capacity # Vessel capacity in TEU per firm, 
        #self.max_number_of_vessel = max_number_of_vessel # # of vessels per firms
        self.design_speed = design_speed  #design speed of the vessel in knots
        self.min_speed = min_speed #min vessel speed
        self.max_speed = max_speed #max vessel speed         
        self.main_engine_power =  main_engine_power  # PS_m : main engine power in kW
        self.aux_engine_power =  aux_engine_power  # auxiliary engine power [kW] 
        self.fix_cost = fix_cost #Daily cost of vessel (USD/Day) $25,000
        self.fuel_type = fuel_type
        self.SFOC_M = SFOC_M
        self.SFOC_A = SFOC_A
        self.market_share = market_share # the firm's market share 
        #self.bau_emission = bau_emission
         
   
    # Firm methods # Reupdate to make market share as in input
    #def get_market_share(self , market_instance):
        #'''return market share of the firm accoding to it s capacity'''
        #self.transport_capacity = np.multiply(self.max_number_of_vessel , self.capacity)
        #self.market_share= np.true_divide(self.transport_capacity, market_instance.market_capacity)
        #return self.market_share
        
    #def get_firm_param(self, market_instance):
        #self.psy = np.multiply (self.market_share , market_instance.market_psy)
        #return self.psy
    
    def get_firm_demand(self, market_instance):
        self.firm_demand = np.multiply (self.market_share , market_instance.market_demand)
        #print("firme demand", self.firm_demand)
        return self.firm_demand 
    
    def update_vessel_speed(self, operational_speed):
        self.operational_speed = np.round(operational_speed, 2) 
        #print(self.operational_speed)
        return self.operational_speed 
    
    def get_min_number_of_vessel(self, market_instance):
        self.time_at_sea = np.true_divide(market_instance.distance,  self.operational_speed) #returns a vector 
        #print( " operational_speed", self.operational_speed)
        #print( " self.time_at_sea", self.time_at_sea)
        self.voyage_time = self.time_at_sea + market_instance.port_time #returns a vector 
        self.number_of_trips_to_meet_demand = np.ceil(np.true_divide(self.firm_demand, self.capacity)) #returns a vector
        self.max_trips_per_vessel = np.floor(np.true_divide(market_instance.annual_working_time, self.voyage_time)) #returns a vector
        self.min_number_of_vessel_to_meet_demand = np.true_divide(self.number_of_trips_to_meet_demand, self.max_trips_per_vessel) #returns a vector
        #print("max_trips_per_vessel", self.number_of_trips_to_meet_demand)
        #print("min_number_of_vessel_to_meet_demand", self.min_number_of_vessel_to_meet_demand)
        return self.min_number_of_vessel_to_meet_demand, self.number_of_trips_to_meet_demand, self.time_at_sea 
    
    def get_number_of_vessel(self):
        self.number_of_vessel = np.ceil(self.min_number_of_vessel_to_meet_demand) #returns a vector
        #print("self.number_of_vessel", self.number_of_vessel)
        return self.number_of_vessel
            
    def get_ship_energy_efficiency(self, market_instance):
        '''ship effici = SFPC * ENGINE POWER * ENGINE LOAD)* (Vds)^-3 '''
        self.main_fuel_parameter = self.SFOC_M * market_instance.eng_load_main * self.main_engine_power * 10**(-6)#self.main_fuel_parameter =np.multiply(np.multiply(np.multiply(self.SFOC_M, market_instance.eng_load_main),self.main_engine_power), 10**(-6))
        self.ship_energy = np.multiply(self.main_fuel_parameter, np.power(self.design_speed, -3) )  
        #print("self.ship_energy", self.ship_energy)
        return self.ship_energy
        
    def get_main_fuel_cons(self, market_instance):
        ''' fuel consu = ship eff * d * V^2 * # of trips to meet demand'''
        self.moo_1 = np.multiply(market_instance.distance, self.ship_energy)
        self.moo_2 = np.power(self.operational_speed, 2)
        self.moo_3 = np.multiply(self.moo_1, self.moo_2)
        self.main_fuel_cons = np.multiply(self.moo_3 , self.number_of_trips_to_meet_demand)
        #print("self.main_fuel_cons ", self.main_fuel_cons )
        return self.main_fuel_cons #returns a vector
        
    def get_aux_fuel_cons(self, market_instance): 
        self.aux_fuel_parameter = np.multiply(np.multiply(np.multiply( self.SFOC_A, market_instance.eng_load_aux), self.aux_engine_power),  10**(-6)) #self.aux_fuel_parameter = self.SFOC_A * market_instance.eng_load_aux * self.aux_engine_power * 10**(-6)
        self.aux_fuel_cons =  np.true_divide((self.aux_fuel_parameter * self.number_of_trips_to_meet_demand * market_instance.distance), self.operational_speed)        
        #print("self.aux_fuel_cons",self.aux_fuel_cons)
        return self.aux_fuel_cons #returns a vector   
    
        
    def get_fuel_cost(self, market_instance):
        self.aux_fuel_cost =np.multiply(self.aux_fuel_cons, market_instance.fuel_data['MGO']['price'])  
        self.main_fuel_cost = np.multiply(self.main_fuel_cons, market_instance.fuel_data[self.fuel_type]['price'])   
        self.fuel_cost = self.main_fuel_cost + self.aux_fuel_cost 
        #print("self.fuel_cost", self.fuel_cost)
        return self.fuel_cost #returns a vector
    
    
    def get_firm_carbon_emission(self, market_instance):
        self.carbon_aux_emision_factor = market_instance.fuel_data['MGO']['carbon_factor']
        self.carbon_aux_emissions = np.multiply(self.carbon_aux_emision_factor, self.aux_fuel_cons )  
        self.carbon_main_emision_factor =  market_instance.fuel_data[self.fuel_type]['carbon_factor']
        self.carbon_main_emissions =np.multiply(self.carbon_main_emision_factor, self.main_fuel_cons)  
        self.carbon_emissions = self.carbon_main_emissions + self.carbon_aux_emissions 
        #print("self.carbon_emissions", self.carbon_emissions )
        return self.carbon_emissions
    
    def get_firm_sulfur_emission(self, market_instance):
        self.sulfur_aux_emision_factor = market_instance.fuel_data['MGO']['sulfur_factor']
        self.sulfur_aux_emissions = np.multiply(self.sulfur_aux_emision_factor, self.aux_fuel_cons)  
        self.sulfur_main_emision_factor =  market_instance.fuel_data[self.fuel_type]['sulfur_factor']
        self.sulfur_main_emissions = np.multiply(self.sulfur_main_emision_factor, self.main_fuel_cons)  
        self.sulfur_emissions = self.sulfur_main_emissions + self.sulfur_aux_emissions 
        #print("self.sulfur_emissions", self.sulfur_emissions)
        return self.sulfur_emissions      
    
    
    def get_firm_operating_cost(self):
        self.operating_cost = np.multiply(self.fix_cost, self.number_of_vessel)  #Fixed Cost
        return self.operating_cost
        
            
    def get_total_cost(self):
        self.total_cost = self.operating_cost + self.fuel_cost
        #print("self.total_cost", self.total_cost)
        return self.total_cost #returns a vector    
     
                 
    def get_revenue(self,market_instance):
        self.revenue = np.multiply(self.firm_demand, market_instance.freight_rate)
        #print("self.revenue", self.revenue)
        return self.revenue #returns a vector
    
    def get_period_profits(self):
        self.period_profit = self.revenue - self.total_cost
        #print("self.period_profit", self.period_profit)
        return self.period_profit #returns a vector

# 1. Simulation Parameters:

In [6]:
####################################### 1. Firm & Market specific Parameters ###########################################
                    #===========================>  Firm 1 Attributes <==========================#
index_1 = 1
capacity_1 = np.array([14000]) 
#max_number_of_vessel_1= np.array([100]) 
design_speed_1 = np.array([25.0]) #design speed of the vessel isn knots
min_speed_1 = 12.0
max_speed_1 = 28.0
main_engine_power_1 =np.array([89700])  # main engine power in kW
aux_engine_power_1 =np.array([14000])  # auxiliary engine power [kW] 
fix_cost_1 = np.array([50000*365])   #Daily cost of vessel (USD/Day) $25,000
fuel_type_1 = "HFO"
SFOC_M_1 = 175
SFOC_A_1 = 32
market_share_1 = 0.2 #UPDATE Firm 1's market share 
                    #===========================>  Firm 2 Attributes <==========================#
index_2 = 2
capacity_2 = np.array([12000])
#max_number_of_vessel_2 = np.array([100])
design_speed_2 = np.array([25.0])  #design speed of the vessel in knots
min_speed_2 = 12.0
max_speed_2 = 28.0
main_engine_power_2 = np.array([82100]) # main engine power in kW
aux_engine_power_2 =np.array([14000])   # auxiliary engine power [kW] 
fix_cost_2 =np.array([45862*365]) #Daily cost of vessel (USD/Day) $25,000
fuel_type_2 = "HFO"
SFOC_M_2 = 159
SFOC_A_2 = 28
market_share_2 = 0.2 #UPDATE Firm 2's market share 
                    #===========================>  Firm 3 Attributes <==========================#
index_3 = 3
capacity_3 = np.array([10000])
#max_number_of_vessel_3 = np.array([100])
design_speed_3 = np.array([25.0])  #design speed of the vessel in knots
min_speed_3 = 12.0
max_speed_3 = 28.0
main_engine_power_3 = np.array([74000]) # main engine power in kW
aux_engine_power_3 =np.array([12000])   # auxiliary engine power [kW] 
fix_cost_3 =np.array([41756*365]) #Daily cost of vessel (USD/Day) $25,000
fuel_type_3 = "HFO"
SFOC_M_3 = 143
SFOC_A_3 = 24
market_share_3 = 0.2#UPDATE Firm 3's market share 
                    #===========================>  Firm 4 Attributes <==========================#
index_4 = 4
capacity_4 = np.array([8000])
#max_number_of_vessel_4 = np.array([100])
design_speed_4 = np.array([25.0])  #design speed of the vessel in knots
min_speed_4 = 12.0
max_speed_4 = 28.0
main_engine_power_4 = np.array([68500]) # main engine power in kW
aux_engine_power_4 =np.array([12000])   # auxiliary engine power [kW] 
fix_cost_4 =np.array([37618*365]) #Daily cost of vessel (USD/Day) $25,000
fuel_type_4 = "HFO"
SFOC_M_4 = 133
SFOC_A_4 = 24
market_share_4 = 0.2 #UPDATE Firm 3's market share 
                    #===========================>  Firm 5 Attributes <==========================#
index_5 = 5
capacity_5 = np.array([6000])
#max_number_of_vessel_5 = np.array([100])
design_speed_5 = np.array([25.0])  #design speed of the vessel in knots
min_speed_5 = 12.0
max_speed_5 = 28.0
main_engine_power_5 = np.array([57100]) # main engine power in kW
aux_engine_power_5 =np.array([12900])   # auxiliary engine power [kW] 
fix_cost_5 =np.array([33466*365]) #Daily cost of vessel (USD/Day) $25,000
fuel_type_5 = "HFO"
SFOC_M_5 = 114
SFOC_A_5 = 26
market_share_5 = 0.2 #UPDATE Firm 3's market share 


                    #===========================> Market Attributes <==========================#
number_of_firms = 5
pollution_decay_parameter =  np.array([1])
pollution_damage_parameter =  np.array([1.5])
year  = MarketConfig.year
income_elasticity = np.array([0.8]) #Constant income elasticity #IMF
price_elasticity = np.array([-0.7])   #Constant own price elasticity#IMF
freight_rate = np.array([822]) ## in US$/TEU
fuel_data = pd.DataFrame(np.array([[422.50, 525.50, 597.00], 
                                [3.114,3.206,3.206],
                                [0.07,0.01,0.002]]),
                         columns=['HFO', 'ULSFO', 'MGO'],
                         index = ['price', 'carbon_factor', 'sulfur_factor'])
discount = np.array([0.02])  ##3  to 5 % transport canada ,#to be updated using the Ramsey rule,#pure rate of time preference of 1.5% + growth rate of consumption g * rate of risk conversion , an elasticity value of 2

# 2. Set up firm and market objects 

In [7]:
""" This cell needs to run once throughout the kernel"""
####################################### 2.Simulation Game Initialisation ###########################################
firm_1 = Firm(index_1, capacity_1, design_speed_1, 
                 min_speed_1, max_speed_1, main_engine_power_1, aux_engine_power_1,
                 fix_cost_1, fuel_type_1, SFOC_M_1, SFOC_A_1, market_share_1) 

firm_2 = Firm(index_2, capacity_2, design_speed_2, 
                 min_speed_2, max_speed_2, main_engine_power_2, aux_engine_power_2,
                 fix_cost_2, fuel_type_2, SFOC_M_2, SFOC_A_2, market_share_2) 

firm_3 = Firm(index_3, capacity_3,  design_speed_3, 
                 min_speed_3, max_speed_3, main_engine_power_3, aux_engine_power_3,
                 fix_cost_3, fuel_type_3,SFOC_M_3, SFOC_A_3, market_share_3) 

firm_4 = Firm(index_4, capacity_4,  design_speed_4, 
                 min_speed_4, max_speed_4, main_engine_power_4, aux_engine_power_4,
                 fix_cost_4, fuel_type_4,SFOC_M_4, SFOC_A_4, market_share_4) 


firm_5 = Firm(index_5, capacity_5,  design_speed_5, 
                 min_speed_5, max_speed_5, main_engine_power_5, aux_engine_power_5,
                 fix_cost_5, fuel_type_5,SFOC_M_5, SFOC_A_5, market_share_5) 


game_config = MarketConfig(number_of_firms, pollution_decay_parameter, pollution_damage_parameter, 
                           income_elasticity, price_elasticity, freight_rate, 
                           fuel_data, discount)

####################################### 3.Get market capacity #################################
#for  firm_object in Firm._registry:
#    market_capacity_sim = game_config.get_market_capacity(firm_object)

# 3. Demand

In [8]:
## Get freight ratio for passethrough
beta_sim = game_config.get_freight_rate_ratio()

# Get market demand  
#market_demand_sim, gdp_sim, market_psy_sim = game_config.get_market_demand()
market_demand_sim = game_config.get_market_demand()

# 7.Get firm level demand  
firm_1_demand_sim = firm_1.get_firm_demand(game_config)
#print("firm 1 demand", firm_1_demand_sim)
    
# 9.Get ship energy efficiency  
firm_1_ship_energy_sim = firm_1.get_ship_energy_efficiency(game_config)

# 7.Get firm level demand  
firm_2_demand_sim = firm_2.get_firm_demand(game_config)
#print("firm 2 demand", firm_2_demand_sim)


# 9.Get ship energy efficiency  
firm_2_ship_energy_sim = firm_2.get_ship_energy_efficiency(game_config)
#print("firm 2 ship energy", firm_2_ship_energy_sim)

# 7.Get firm level demand  
firm_3_demand_sim = firm_3.get_firm_demand(game_config)
#print("firm 3 demand", firm_3_demand_sim)

# 9.Get ship energy efficiency  
firm_3_ship_energy_sim = firm_3.get_ship_energy_efficiency(game_config)


# 7.Get firm level demand  
firm_4_demand_sim = firm_4.get_firm_demand(game_config)
#print("firm 1 demand", firm_4_demand_sim)
    
# 9.Get ship energy efficiency  
firm_4_ship_energy_sim = firm_4.get_ship_energy_efficiency(game_config)


# 7.Get firm level demand  
firm_5_demand_sim = firm_5.get_firm_demand(game_config)
#print("firm 1 demand", firm_5_demand_sim)
    
# 9.Get ship energy efficiency  
firm_5_ship_energy_sim = firm_5.get_ship_energy_efficiency(game_config)


In [9]:
market_demand_sim

array([ 7000000.        ,  7200000.        ,  7200000.        ,
        7800000.        ,  8104290.37379516,  8336865.5491866 ,
        8562867.59239633,  8788190.97193528,  9019443.51303361,
        9256781.23570516,  9500364.26547593,  9750356.94141652,
       10006927.92701742, 10270250.32398178, 10540501.78901267,
       10817864.65367329, 11102526.04740139, 11394678.02376047,
       11694517.69001338, 12002247.34010533, 12318074.59114637,
       12642212.52348514, 12974879.82446861, 13316300.9359847 ,
       13666706.2058873 ])

# 4. GA set up 

In [10]:
#====> Iinitialisation of chromo and population objects 
Chromo =  float
my_target = float

Population = List[Chromo]
#====> Iinitialisation of function objects; Define the Objects; They allow you to pass in only what you need during the evol function 
PopulateFunc = Callable[[], Population]
VesselProc = Callable[[Chromo, Firm], Tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray, np.ndarray , np.ndarray ,np.ndarray, np.ndarray, np.ndarray]]
Integrated_Solution_Combo_Func = Callable[[dict, list , np.ndarray, np.ndarray,] , dict]

SelectionFunc = Callable[[np.ndarray, np.ndarray], Tuple[Chromo, Chromo]]  #takes a population and a fitness fn to select 2 solutions to be the parents of our next generaation solution
CrossoverFunc = Callable[[Chromo, Chromo], Tuple[Chromo, Chromo]] #takes 2 genomes and returns 2 new genomes 
MutationFunc = Callable[[Chromo], Chromo] #takes 1 genome and sometimes returns a modified one
FitnessFunc = Callable[[Chromo, my_target], list] #int to refelct max # of vessel


### 1) GA specific functions 

In [11]:
#inti population
chromo_init = pd.read_csv('init_chromo.csv')
chromo_init
chromo_init['population'] = chromo_init.values.tolist()
my_init_population = np.array([chromo_init['population'][_] for _ in range(64)])
my_init_population =np.round(my_init_population, 2)
my_init_population

array([[12.  , 12.  , 12.  , 12.  , 12.  ],
       [12.  , 12.  , 12.  , 12.  , 16.67],
       [12.  , 12.  , 12.  , 12.  , 16.67],
       [12.  , 12.  , 12.  , 12.  , 21.78],
       [12.  , 12.  , 12.  , 16.67, 12.  ],
       [12.  , 12.  , 12.  , 16.67, 16.67],
       [12.  , 12.  , 12.  , 16.67, 16.67],
       [12.  , 12.  , 12.  , 16.67, 21.78],
       [12.  , 12.  , 16.67, 12.  , 12.  ],
       [12.  , 12.  , 16.67, 12.  , 16.67],
       [12.  , 12.  , 16.67, 12.  , 16.67],
       [12.  , 12.  , 16.67, 12.  , 21.78],
       [12.  , 12.  , 16.67, 16.67, 12.  ],
       [12.  , 12.  , 16.67, 16.67, 16.67],
       [12.  , 12.  , 16.67, 16.67, 16.67],
       [12.  , 12.  , 16.67, 16.67, 21.78],
       [12.  , 16.67, 12.  , 12.  , 12.  ],
       [12.  , 16.67, 12.  , 12.  , 16.67],
       [12.  , 16.67, 12.  , 12.  , 16.67],
       [12.  , 16.67, 12.  , 12.  , 21.78],
       [12.  , 16.67, 12.  , 16.67, 12.  ],
       [12.  , 16.67, 12.  , 16.67, 16.67],
       [12.  , 16.67, 12.  , 16.

In [12]:
# #======> Generate Chromo
def generate_chromo(target:my_target, fitness_func : FitnessFunc) :
    while True:
        chromo = np.random.uniform(firm_1.min_speed, 22.0, np.size(Firm._registry) )  #firm_1.max_speed
        #print(chromo)
        if fitness_func(chromo, target )[0] != 0:
            #print("chrom invalid", chromo)
            break
    return chromo  


# #======>
# def generate_chromo(lower_bound:float, upper_bound:float, fitness_func: FitnessFunc ) -> Chromo:
#     while True:
#         chromo = np.random.uniform(lower_bound, upper_bound)   
#         print("geneerate stuck")
#         if fitness_func(chromo)[0] != 0:
#             break
#     return chromo                    
              


#======> Generate Population
def generate_population(target:my_target, fitness_func: FitnessFunc, size: int ):
    '''generate the population of the firm s vessel speed based on our coev choice param population size'''
    new_size = size -64
    left_population = np.array([generate_chromo(target, fitness_func) for _ in range(new_size)])
    return  np.round(np.concatenate((my_init_population , left_population), axis=0),2)

#=====> Select Parent Chromo 

def selection_pair (chromo, weights): 
    '''randomly select 2 chromo from the population based on their weights, ie , solution fitness''' 
    #print(chromo)
    #print(weights)
    weights = np.asarray(weights).astype('float64')
    weights = weights / np.sum(weights)
    selected_index = np.random.choice(np.arange(np.size(weights)),size = 2, replace=False,p= weights)
    #print("chromo[selected_index[0]]", chromo[selected_index[0]])
    #print("chromo[selected_index[1]", chromo[selected_index[1]])
    return chromo[selected_index[0]],chromo[selected_index[1]]



def BLX_alpha_crossover (a: Chromo, b: Chromo,  sim_game) :
    child = np.zeros(np.size(Firm._registry))
    lower = np.zeros(np.size(Firm._registry))
    upper = np.zeros(np.size(Firm._registry))
    for i in range(np.size(Firm._registry)): 
        lower[i] = (min(a[i],b[i])) - ( (0.5) * (max(a[i],b[i]) - min (a[i],b[i])))
        upper[i] = (max(a[i],b[i])) + ( (0.5) * (max(a[i],b[i]) - min (a[i],b[i])))
        lower[i] = firm_1.min_speed if (lower[i] < firm_1.min_speed) else lower[i]
        upper[i]= firm_1.max_speed if (upper[i] > firm_1.max_speed) else upper[i]
        child[i] = np.random.uniform (lower[i], upper[i])
    
    return np.round(child,2)

#=====>#mutation prob = 0.5
def mutation(chromo: Chromo, sim_game, variance: float, probability: float = 0.1 ) : 
    mutated_chromo = np.zeros(np.size(Firm._registry))
    #print("chromo",chromo)
    for i in range(np.size(Firm._registry)): 
        mutated_chromo[i] = np.random.normal(chromo[i], variance, 1) if random.random() > probability else chromo[i]
        #print("mutated_chromo ",mutated_chromo[i])
        mutated_chromo[i] = firm_1.min_speed  if mutated_chromo[i] < firm_1.min_speed  else mutated_chromo[i]
        mutated_chromo[i] = firm_1.max_speed if mutated_chromo [i] > firm_1.max_speed else mutated_chromo[i]
    return np.round(mutated_chromo,2)



#====> 
def fitness_similarity_chech(max_fitness, number_of_similarity):
    result = 0
    similarity = 0
    for n in range(len(max_fitness)-1):
        if np.round(max_fitness[n], 3 ) == np.round(max_fitness[n+1],3):
            similarity += 1
        else:
            similarity = 0
    if similarity == number_of_similarity-1:
        result = 1
    return result



### 2. Vessel Operations and Fitness Evaluation

In [13]:
def vessel_operation_procedure (chromo: Chromo, firm: Firm, sim_game: MarketConfig): 
    
    operational_speed_ = np.repeat (firm.update_vessel_speed(chromo),sim_game.T) # firm.update_vessel_speed(chromo)
    #print("operational_speed", operational_speed_)
    min_number_vessels_sim , trips_to_meet_demand_sim, time_at_sea_sim  = firm.get_min_number_of_vessel(sim_game)
    #print(min_number_vessels_sim, trips_to_meet_demand_sim, time_at_sea_sim )
    number_vessels_sim  = firm.get_number_of_vessel()
    #print("number_vessels_sim", number_vessels_sim)
    
    #if (number_vessels_sim > firm.max_number_of_vessel).any():
        #true 
        #return np.zeros(9) #np.array([0,..,0])
    #else:
    main_fuel_con_sim = firm.get_main_fuel_cons(sim_game)
    #print("main_fuel_con_sim", main_fuel_con_sim)
    aux_fuel_con_sim = firm.get_aux_fuel_cons(sim_game)
    #print("aux_fuel_con_sim", aux_fuel_con_sim)
    fuel_cost_sim = firm.get_fuel_cost(sim_game) 
    #fixed cost
    fixed_cost_sim = firm.get_firm_operating_cost()
    
    #print("fuel_cost_sim", fuel_cost_sim)
    total_cost_sim = firm.get_total_cost()
    #print("total_cost_sim", total_cost_sim)
    revenue_sim = firm.get_revenue(sim_game)
    #print("revenue_sim", revenue_sim)
    profit_vector_sim = firm.get_period_profits()
    #print("profit_vector_sim", profit_vector_sim)
     
    
    carbon_emissions_sim = firm.get_firm_carbon_emission(sim_game) 
    #print("carbon_emissions_sim", carbon_emissions_sim)
    sulfur_emissions_sim = firm.get_firm_sulfur_emission(sim_game)
    
    return number_vessels_sim, main_fuel_con_sim, aux_fuel_con_sim, fuel_cost_sim, fixed_cost_sim, total_cost_sim , revenue_sim ,profit_vector_sim, carbon_emissions_sim, sulfur_emissions_sim

In [14]:
# objective function
def fitness(
            chromo_array, 
          target:my_target, 
            sim_game,           
            carbon_pollution_decay_parameter,
            sulfur_pollution_decay_parameter,
            carbon_pollution_damage_parameter,
            sulfur_pollution_damage_parameter,
            initial_carbon_tax_sim,
            initial_sulfur_tax_sim,
            taxation_scheme_rate_sim):
    """get 3 chromos and returns payoffs"""
    
    
    # discount rate
    discount_multiplier = np.power( 1+ sim_game.discount_rate, - np.arange(1,MarketConfig.T + 1))
    
    # vessel speed operations
    operations = [vessel_operation_procedure (chromo_array[firm.index -1 ], firm, game_config ) for  firm in  Firm._registry ] 
    number_vessels_sim = [operations[firm.index -1 ][0] for  firm in  Firm._registry ] 
    
    main_fuel_con_sim = [operations[firm.index -1 ][1] for  firm in  Firm._registry ] 
    aux_fuel_con_sim = [operations[firm.index -1 ][2] for  firm in  Firm._registry ] 
    fuel_cost_sim =[operations[firm.index -1 ][3] for  firm in  Firm._registry ] 
    fixed_cost_sim =[operations[firm.index -1 ][4] for  firm in  Firm._registry ] 
    total_cost_sim  =[operations[firm.index -1 ][5] for  firm in  Firm._registry ] 
    revenue_sim  =[operations[firm.index -1 ][6] for  firm in  Firm._registry ] 


    profits =  [operations[firm.index -1 ][7] for  firm in  Firm._registry ] 
    carbon_emissions =   [operations[firm.index -1 ][8] for  firm in  Firm._registry ] 
    #print("carbon_emissions",carbon_emissions)
    sulfur_emissions =   [operations[firm.index -1 ][9] for  firm in  Firm._registry ]   
    #print("sulfur_emissions", sulfur_emissions)
    
    #Indutry level 
    industry_carbon_emission = (np.sum(carbon_emissions, axis = 0)).reshape(-1)
    industry_sulfur_emission = (np.sum(sulfur_emissions, axis = 0)).reshape(-1)
    #print ("industry_carbon_emission", industry_carbon_emission)
    #print ("industry_sulfur_emission", industry_sulfur_emission)

    
    if( np.sum(industry_carbon_emission) < target  ): #(np.all( np.sum(industry_carbon_emission) < coalition_target  )):
        #pollution stock 
        carbon_pollution_stock=np.zeros(MarketConfig.T,  dtype=object) #[0]*MarketConfig.T
        sulfur_pollution_stock= np.zeros(MarketConfig.T,  dtype=object) #[0]*MarketConfig.T

        for my_period_index in range (MarketConfig.T-1):
            carbon_pollution_stock [my_period_index + 1] = (1- carbon_pollution_decay_parameter ) * carbon_pollution_stock[my_period_index ] + industry_carbon_emission[my_period_index]
            sulfur_pollution_stock [my_period_index + 1] = (1- sulfur_pollution_decay_parameter ) * sulfur_pollution_stock[my_period_index ] + industry_sulfur_emission[my_period_index]

        #print("carbon_pollution_stock", carbon_pollution_stock)
        #print("sulfur_pollution_stock", sulfur_pollution_stock)

        #dynamic tax
        carbon_dynamic_tax = np.zeros(MarketConfig.T,  dtype=object) #[0]*MarketConfig.T 
        sulfur_dynamic_tax = np.zeros(MarketConfig.T,  dtype=object) #[0]*MarketConfig.T

        carbon_dynamic_tax[0] = initial_carbon_tax_sim
        sulfur_dynamic_tax[0] = initial_sulfur_tax_sim
        for my_period_index in range (MarketConfig.T-1):
            carbon_dynamic_tax[my_period_index+1] = initial_carbon_tax_sim + (taxation_scheme_rate_sim * carbon_pollution_stock[my_period_index +1 ])
            sulfur_dynamic_tax[my_period_index+1] = initial_sulfur_tax_sim + (taxation_scheme_rate_sim * sulfur_pollution_stock[my_period_index +1 ])


        #print("carbon_dynamic_tax", carbon_dynamic_tax)
        #print("sulfur_dynamic_tax", sulfur_dynamic_tax)

        #### Get the industry s damages 
        industry_carbon_damages = 0.5* carbon_pollution_damage_parameter * np.power( carbon_pollution_stock,2)
        industry_sulfur_damages = 0.5* sulfur_pollution_damage_parameter * np.power( sulfur_pollution_stock,2)

        #print("industry_carbon_damages", industry_carbon_damages)
        #print("industry_sulfur_damages", industry_sulfur_damages)




        #### Get Abatement benefits 
        #abatement_benefits =   BAU_carbon_damages - industry_carbon_damages


        ### Get NPV for each firm 
            #print("tac", carbon_dynamic_tax )
            #print("carbon_emissions_sim ]", carbon_emissions_sim["firm" + str (firm.index)])
            #print ("check",  carbon_emissions_sim["firm" + str (firm.index)] * carbon_dynamic_tax )



        #abatement_benefits_firm = [np.multiply( abatement_benefits ,  firm.market_share  ) for firm in Firm._registry ] 

        carbon_policy_cost = [np.multiply(carbon_emissions[firm.index - 1]  , carbon_dynamic_tax ) for firm in Firm._registry ] 
        #print("carbon_policy_cost",carbon_policy_cost)
        sulfur_policy_cost = [np.multiply(sulfur_emissions[firm.index - 1]  , sulfur_dynamic_tax ) for firm in Firm._registry ]
        #print("sulfur_policy_cost",sulfur_policy_cost)
        policy_cost = [carbon_policy_cost[firm.index - 1] + sulfur_policy_cost[firm.index - 1] for firm in Firm._registry ] 



        #print("policy_cost",policy_cost)
        period_profit_witout_benefits = [profits[firm.index - 1]  -  policy_cost[firm.index - 1] for firm in Firm._registry ]  

        period_profit = period_profit_witout_benefits#[period_profit_witout_benefits[firm.index - 1]  +  abatement_benefits_firm[firm.index - 1] for firm in Firm._registry ]  

        #print("period_profit", period_profit)
        discounted_period_profit = [np.multiply( discount_multiplier, period_profit[firm.index - 1]) for firm in Firm._registry]
        #print("discounted_period_profit", discounted_period_profit)

        payoff_vector=[np.sum(discounted_period_profit[firm.index - 1]) for firm in Firm._registry] 
        #print("payoff for firms", payoff_vector)

        ### sum the payoff for across all firms 
        payoff = np.sum(payoff_vector)
        #print("coalition payoff", payoff)
        return payoff, payoff_vector, number_vessels_sim, main_fuel_con_sim, aux_fuel_con_sim, fuel_cost_sim, fixed_cost_sim, total_cost_sim , revenue_sim ,profits, carbon_emissions, sulfur_emissions
    
    else:
        return np.zeros(10) #np.array([-9, -9, -9, -9])



In [15]:
def run_evolution(populate_func: PopulateFunc,
                  fitness_func : FitnessFunc, 
                  mutation_func: MutationFunc,
                  size,
                  sim_game,
                  target,
                  generation_limit,
                  number_of_similarity,
                  selection_func: SelectionFunc = selection_pair,
                               crossover_func: CrossoverFunc = BLX_alpha_crossover):

                 
    """The evolutionnary main loop""" 
    #1. Genrate the 1st generation: Initial generation/population
    population = populate_func(target,fitness_func) 
    
    #2 Simulate until you reach generation limit,
    i = 0
    progress = pd.DataFrame(columns=['Generation', 'Vessel Speed', 'coalition Net present value' ,
                                     'Firm Net present value', 
                                'Number of vessels', 
                                     'main_fuel_con_sim', 'aux_fuel_con_sim',  'fuel_cost_sim',"fixed_cost_sim",
                                    'total_cost_sim','revenue_sim','profit_vector_sim', "carbon_emissions_sim",
                                    "sulfur_emissions_sim"])
        
        
    
    population_ledger =  pd.DataFrame(columns=['Generation', 
                                               'Vessel Speed'])
    
    # Create the next generation 
    for i in range(generation_limit):
        #Step 1 : Sort solutions in the population of generation i based on fitness to select the best 2 solutions;
        ## THE first 2 observations
        population = sorted(population, key=lambda chromo: fitness_func(chromo, target)[0], reverse=True) 
        #print(population)
        ##payoff, number_vessels_sim, main_fuel_con_sim, aux_fuel_con_sim, fuel_cost_sim, total_cost_sim , revenue_sim ,profit_vector_sim
        ## remove emissions from BAU 
        progress = progress.append({'Generation': i,
                          'Vessel Speed': population[0],
                          'coalition Net present value': fitness_func( population[0],target)[0],
                          'Firm Net present value':fitness_func( population[0],target)[1],
                          'Number of vessels': fitness_func(population[0], target)[2] , 
                          'main_fuel_con_sim': fitness_func(population[0], target)[3],
                          'aux_fuel_con_sim': fitness_func(population[0], target)[4], 
                          'fuel_cost_sim': fitness_func(population[0], target)[5], 
                          "fixed_cost_sim" : fitness_func(population[0], target)[6], 
                          'total_cost_sim' : fitness_func(population[0], target)[7], 
                          'revenue_sim' : fitness_func(population[0], target)[8],
                          'profit_vector_sim': fitness_func(population[0], target)[9],
                                   "carbon_emissions_sim":fitness_func(population[0],target)[10] , 
                                    "sulfur_emissions_sim" : fitness_func(population[0],target)[11]
                                   }, ignore_index=True) 
        
        
        

        population_ledger = population_ledger.append({'Generation': i,
                                                  'Vessel Speed': population}, ignore_index=True) 
        
        
        #Similarity check
        if fitness_similarity_chech(progress['coalition Net present value'], number_of_similarity) == 1:
            break
#         print(progress)
#         print (i)
#         print("Best vessel speed in the", i, "th generation is" ,population[0] ) 
#         print("Best fitness in the", i, "th generation is", fitness_func(population[0]))
    
              
        #Step 2: Implement elitism and pick the top 2 chromo(speed) in the population based on fitness for the next gernation 
        next_generation = population[0:2] #pick the top 2      
        
        #print("next_generation", next_generation)
        max = np.sum([fitness_func( chromo, target)[0] for chromo in population])
        #print("max", max)
        weights= np.hstack([fitness_func(chromo, target)[0] / max for chromo in population])
        weights[weights<0] = 0 #in case the NPV <0
        
        

        
        #Step 3: j in range 24 
        for j in range(int(len(population) / 2) - 1):
            #Step 1: Selection
            #print("weights", weights)
            parents = selection_func(population, weights = weights)
            
            
            

            #print("parent are", parents) 
            
            #Step 2: Crossover
            #offspring_a, offspring_b = crossover_func(parents[0], parents[1])
            
            #5. Crossover function: 
            #Generate 2 offspring by a crossover operation. 
            offspring_a = crossover_func(parents[0], parents[1], sim_game )
            offspring_b = crossover_func(parents[0], parents[1], sim_game)
            #print("offspring a are :" , offspring_a)
            #print("offspring b are :" , offspring_b)

            

            #Step 3: mutation 
            offspring_a = mutation_func(offspring_a)
            offspring_b = mutation_func(offspring_b)
            #print("offspring a ", offspring_a) 
            #print("offspring b", offspring_b) 
            
            #Step 4: Next generation 
            next_generation += [offspring_a, offspring_b]
            
        #print("next generation",i, "is", next_generation)     
        #print(i)
        population = next_generation #update current population with our next generation and start into the next round of the algo by sorting the population and checking if we reached our fitness limit 
    
    #Sort the population one last time in case we run out of generation  
    population = sorted(population, key=lambda chromo: fitness_func(chromo,target )[0], reverse=True)
    progress = progress.append({'Generation': i,
                          'Vessel Speed': population[0] ,
                          'coalition Net present value': fitness_func( population[0],target)[0],
                          'Firm Net present value':fitness_func( population[0],target)[1],
                          'Number of vessels': fitness_func(population[0],target)[2] , 
                          'main_fuel_con_sim': fitness_func(population[0],target)[3],
                          'aux_fuel_con_sim': fitness_func(population[0], target)[4], 
                          'fuel_cost_sim': fitness_func(population[0], target)[5], 
                          "fixed_cost_sim" : fitness_func(population[0], target)[6], 
                          'total_cost_sim' : fitness_func(population[0], target)[7], 
                          'revenue_sim' : fitness_func(population[0], target)[8],
                          'profit_vector_sim': fitness_func(population[0], target)[9],
                                   "carbon_emissions_sim":fitness_func(population[0], target)[10] , 
                                    "sulfur_emissions_sim" : fitness_func(population[0], target)[11]
                                   }, ignore_index=True) 
        

    population_ledger = population_ledger.append({'Generation': i,
                                                  'Vessel Speed': population}, ignore_index=True) 
    #print(progress)
    return progress, population_ledger


# Run Evolution 

# Set up target 

In [16]:
d_1 = pd.read_csv('emissions_1.csv')[0:MarketConfig.T]
d_2 = pd.read_csv('emissions_2.csv')[0:MarketConfig.T]
d_3 = pd.read_csv('emissions_3.csv')[0:MarketConfig.T]
d_4 = pd.read_csv('emissions_4.csv')[0:MarketConfig.T]
d_5 = pd.read_csv('emissions_5.csv')[0:MarketConfig.T]
industry_carbon_emissions = d_1["carbon"].values + d_2["carbon"].values+ d_3["carbon"].values+ d_4["carbon"].values+ + d_5["carbon"].values


carbon_BAU_1 = np.sum(d_1["carbon"])
carbon_BAU_2 = np.sum(d_2["carbon"])
carbon_BAU_3 = np.sum(d_3["carbon"])
carbon_BAU_4 = np.sum(d_4["carbon"])
carbon_BAU_5 = np.sum(d_5["carbon"])
carbon_BAU_industry = np.sum(industry_carbon_emissions)



In [17]:
sim = {'Emission reduction % target': np.arange(0.01, 0.41, 0.01)} 
sim_df = pd.DataFrame(data=sim)
sim_df['Emission_target_firm_1'] = (1- sim_df['Emission reduction % target'])  * carbon_BAU_1
sim_df['Emission_target_firm_2'] = (1- sim_df['Emission reduction % target'])  * carbon_BAU_2
sim_df['Emission_target_firm_3'] = (1- sim_df['Emission reduction % target'])  * carbon_BAU_3
sim_df['Emission_target_firm_4'] = (1- sim_df['Emission reduction % target'])  * carbon_BAU_4
sim_df['Emission_target_firm_5'] = (1- sim_df['Emission reduction % target'])  * carbon_BAU_5
sim_df['Emission_target_industry'] = (1- sim_df['Emission reduction % target'])  * carbon_BAU_industry



In [18]:
sim_df
sim_df.to_csv("emission_target.csv", index=False)

In [19]:


summary_res = pd.DataFrame(columns=[  'Generation', 'carbon abatement %','carbon target','Vessel Speed', 'Firm Net present value', 
                                'Number of vessels', 'main_fuel_con_sim', 'aux_fuel_con_sim',  'fuel_cost_sim',"fixed_cost_sim",
                                    'total_cost_sim','revenue_sim','profit_vector_sim', "carbon_emissions_sim",
                                    "sulfur_emissions_sim"])

## run from 1% decrease to 60% decrease 


sim_df


,Emission reduction % target,Emission_target_firm_1,Emission_target_firm_2,Emission_target_firm_3,Emission_target_firm_4,Emission_target_firm_5,Emission_target_industry
0,0.01,6.100794e+07,5.934345e+07,5.739305e+07,6.226670e+07,9.951279e+07,3.395239e+08
1,0.02,6.039170e+07,5.874402e+07,5.681333e+07,6.163775e+07,9.850761e+07,3.360944e+08
2,0.03,5.977546e+07,5.814460e+07,5.623360e+07,6.100879e+07,9.750243e+07,3.326649e+08
3,0.04,5.915922e+07,5.754517e+07,5.565387e+07,6.037983e+07,9.649725e+07,3.292353e+08
4,0.05,5.854297e+07,5.694574e+07,5.507414e+07,5.975088e+07,9.549207e+07,3.258058e+08
5,0.06,5.792673e+07,5.634631e+07,5.449442e+07,5.912192e+07,9.448689e+07,3.223763e+08
6,0.07,5.731049e+07,5.574688e+07,5.391469e+07,5.849296e+07,9.348171e+07,3.189467e+08
7,0.08,5.669425e+07,5.514745e+07,5.333496e+07,5.786401e+07,9.247653e+07,3.155172e+08
8,0.09,5.607801e+07,5.454802e+07,5.275523e+07,5.723505e+07,9.147135e+07,3.120877e+08
9,0.10,5.546177e+07,5.394859e+07,5.217550e+07,5.660609e+07,9.046617e+07,3.086581e+08


In [20]:
d_1 = pd.read_csv('emissions_1.csv')[0:MarketConfig.T]
d_2 = pd.read_csv('emissions_2.csv')[0:MarketConfig.T]
d_3 = pd.read_csv('emissions_3.csv')[0:MarketConfig.T]
d_4 = pd.read_csv('emissions_4.csv')[0:MarketConfig.T]
d_5 = pd.read_csv('emissions_5.csv')[0:MarketConfig.T]

industry_carbon_emissions = d_1["carbon"].values + d_2["carbon"].values+ d_3["carbon"].values+ d_4["carbon"].values+ + d_5["carbon"].values
carbon_BAU = np.sum(industry_carbon_emissions)
carbon_BAU

summary_res = pd.DataFrame(columns=[  'Generation', 'carbon abatement %','carbon target','Vessel Speed', 'coalition Net present value' ,'Firm Net present value', 
                                'Number of vessels', 'main_fuel_con_sim', 'aux_fuel_con_sim',  'fuel_cost_sim',"fixed_cost_sim",
                                    'total_cost_sim','revenue_sim','profit_vector_sim', "carbon_emissions_sim",
                                    "sulfur_emissions_sim"])

## run from 1% decrease to 60% decrease 

sim = {'Emission reduction % target': np.arange(0.01, 0.41, 0.01)} 
sim_df = pd.DataFrame(data=sim)
sim_df['Emission target'] = (1- sim_df['Emission reduction % target'])  * carbon_BAU

sim_df


,Emission reduction % target,Emission target
0,0.01,3.395239e+08
1,0.02,3.360944e+08
2,0.03,3.326649e+08
3,0.04,3.292353e+08
4,0.05,3.258058e+08
5,0.06,3.223763e+08
6,0.07,3.189467e+08
7,0.08,3.155172e+08
8,0.09,3.120877e+08
9,0.10,3.086581e+08


In [21]:
# # No target 
# # objective function
# def fitness(
#         chromo_array, 
#         sim_game,       
#         carbon_pollution_decay_parameter,
#         sulfur_pollution_decay_parameter,
#         carbon_pollution_damage_parameter,
#         sulfur_pollution_damage_parameter,
#         initial_carbon_tax_sim,
#         initial_sulfur_tax_sim,
#         taxation_scheme_rate_sim):
#     """get 3 chromos and returns payoffs"""
    
    
#     # discount rate
#     discount_multiplier = np.power( 1+ sim_game.discount_rate, - np.arange(1,MarketConfig.T + 1))
    
#     # vessel speed operations
#     operations = [vessel_operation_procedure (chromo_array[firm.index -1 ], firm, game_config ) for  firm in  Firm._registry ] 
#     number_vessels_sim = [operations[firm.index -1 ][0] for  firm in  Firm._registry ] 
    
#     main_fuel_con_sim = [operations[firm.index -1 ][1] for  firm in  Firm._registry ] 
#     aux_fuel_con_sim = [operations[firm.index -1 ][2] for  firm in  Firm._registry ] 
#     fuel_cost_sim =[operations[firm.index -1 ][3] for  firm in  Firm._registry ] 
#     fixed_cost_sim =[operations[firm.index -1 ][4] for  firm in  Firm._registry ] 
#     total_cost_sim  =[operations[firm.index -1 ][5] for  firm in  Firm._registry ] 
#     revenue_sim  =[operations[firm.index -1 ][6] for  firm in  Firm._registry ] 


#     profits =  [operations[firm.index -1 ][7] for  firm in  Firm._registry ] 
#     carbon_emissions =   [operations[firm.index -1 ][8] for  firm in  Firm._registry ] 
#     #print("carbon_emissions",carbon_emissions)
#     sulfur_emissions =   [operations[firm.index -1 ][9] for  firm in  Firm._registry ]   
#     #print("sulfur_emissions", sulfur_emissions)
    
#     #Indutry level 
#     industry_carbon_emission = (np.sum(carbon_emissions, axis = 0)).reshape(-1)
#     industry_sulfur_emission = (np.sum(sulfur_emissions, axis = 0)).reshape(-1)
#     #print ("industry_carbon_emission", industry_carbon_emission)
#     #print ("industry_sulfur_emission", industry_sulfur_emission)

    
    
#     #pollution stock 
#     carbon_pollution_stock=np.zeros(MarketConfig.T,  dtype=object) #[0]*MarketConfig.T
#     sulfur_pollution_stock= np.zeros(MarketConfig.T,  dtype=object) #[0]*MarketConfig.T

#     for my_period_index in range (MarketConfig.T-1):
#         carbon_pollution_stock [my_period_index + 1] = (1- carbon_pollution_decay_parameter ) * carbon_pollution_stock[my_period_index ] + industry_carbon_emission[my_period_index]
#         sulfur_pollution_stock [my_period_index + 1] = (1- sulfur_pollution_decay_parameter ) * sulfur_pollution_stock[my_period_index ] + industry_sulfur_emission[my_period_index]

#     #print("carbon_pollution_stock", carbon_pollution_stock)
#     #print("sulfur_pollution_stock", sulfur_pollution_stock)

#     #dynamic tax
#     carbon_dynamic_tax = np.zeros(MarketConfig.T,  dtype=object) #[0]*MarketConfig.T 
#     sulfur_dynamic_tax = np.zeros(MarketConfig.T,  dtype=object) #[0]*MarketConfig.T

#     carbon_dynamic_tax[0] = initial_carbon_tax_sim
#     sulfur_dynamic_tax[0] = initial_sulfur_tax_sim
#     for my_period_index in range (MarketConfig.T-1):
#         carbon_dynamic_tax[my_period_index+1] = initial_carbon_tax_sim + (taxation_scheme_rate_sim * carbon_pollution_stock[my_period_index +1 ])
#         sulfur_dynamic_tax[my_period_index+1] = initial_sulfur_tax_sim + (taxation_scheme_rate_sim * sulfur_pollution_stock[my_period_index +1 ])


#     #print("carbon_dynamic_tax", carbon_dynamic_tax)
#     #print("sulfur_dynamic_tax", sulfur_dynamic_tax)

#     #### Get the industry s damages 
#     industry_carbon_damages = 0.5* carbon_pollution_damage_parameter * np.power( carbon_pollution_stock,2)
#     industry_sulfur_damages = 0.5* sulfur_pollution_damage_parameter * np.power( sulfur_pollution_stock,2)

#     #print("industry_carbon_damages", industry_carbon_damages)
#     #print("industry_sulfur_damages", industry_sulfur_damages)




#     #### Get Abatement benefits 
#     #abatement_benefits =   BAU_carbon_damages - industry_carbon_damages


#     ### Get NPV for each firm 
#         #print("tac", carbon_dynamic_tax )
#         #print("carbon_emissions_sim ]", carbon_emissions_sim["firm" + str (firm.index)])
#         #print ("check",  carbon_emissions_sim["firm" + str (firm.index)] * carbon_dynamic_tax )



#     #abatement_benefits_firm = [np.multiply( abatement_benefits ,  firm.market_share  ) for firm in Firm._registry ] 

#     carbon_policy_cost = [np.multiply(carbon_emissions[firm.index - 1]  , carbon_dynamic_tax ) for firm in Firm._registry ] 
#     #print("carbon_policy_cost",carbon_policy_cost)
#     sulfur_policy_cost = [np.multiply(sulfur_emissions[firm.index - 1]  , sulfur_dynamic_tax ) for firm in Firm._registry ]
#     #print("sulfur_policy_cost",sulfur_policy_cost)
#     policy_cost = [carbon_policy_cost[firm.index - 1] + sulfur_policy_cost[firm.index - 1] for firm in Firm._registry ] 



#     #print("policy_cost",policy_cost)
#     period_profit_witout_benefits = [profits[firm.index - 1]  -  policy_cost[firm.index - 1] for firm in Firm._registry ]  

#     #period_profit = [period_profit_witout_benefits[firm.index - 1]  +  abatement_benefits_firm[firm.index - 1] for firm in Firm._registry ]  
#     period_profit =  period_profit_witout_benefits   

#     #print("period_profit", period_profit)
#     discounted_period_profit = [np.multiply( discount_multiplier, period_profit[firm.index - 1]) for firm in Firm._registry]
#     #print("discounted_period_profit", discounted_period_profit)

#     payoff_vector=[np.sum(discounted_period_profit[firm.index - 1]) for firm in Firm._registry] 
#     #print("payoff for firms", payoff_vector)

#     ### sum the payoff for across all firms 
#     payoff = np.sum(payoff_vector)
#     #print("coalition payoff", payoff)
#     return payoff, payoff_vector, number_vessels_sim, main_fuel_con_sim, aux_fuel_con_sim, fuel_cost_sim, fixed_cost_sim, total_cost_sim , revenue_sim ,profits, carbon_emissions, sulfur_emissions
    



In [22]:
## Update with the right firm before size 
for my_index in (sim_df).index:
    print(my_index)
    results, my_population_ledger  = run_evolution(
    
    populate_func = partial(generate_population,
                            size=1000),
     
    fitness_func= partial(fitness,
                           sim_game= game_config,   
                           carbon_pollution_decay_parameter = 1,
                                           sulfur_pollution_decay_parameter=1,
                                           carbon_pollution_damage_parameter=1.5,
                                           sulfur_pollution_damage_parameter=1.5,
                                           initial_carbon_tax_sim =0,
                                           initial_sulfur_tax_sim =0,
                                           taxation_scheme_rate_sim=0), 
  
    
    mutation_func = partial (mutation,
                             sim_game= game_config,
                             variance = 4,
                             probability = 0.05),
    
    
    
    size=1000,
    target  = sim_df["Emission target"][my_index],
    sim_game = game_config,
    generation_limit = 2000,
    number_of_similarity =100)
    
    
    bla = results.iloc[[-1]]
    bla.insert(1, "carbon target",  sim_df["Emission target"][my_index], True)
    bla.insert(1, "carbon abatement %", sim_df["Emission reduction % target"][my_index], True)


    summary_res = summary_res.append(bla)
    #print("done",summary_res )
    
    
    


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39


# POST

In [23]:
summary_res_copy = summary_res.copy(deep=True)
summary_res_copy = summary_res_copy.reset_index(drop=True)
sum_short =  summary_res_copy[["carbon abatement %","carbon target", "Vessel Speed" ,
                               "coalition Net present value" ,"Firm Net present value","carbon_emissions_sim","sulfur_emissions_sim" ]]
sum_short

#summary_res_copy

,carbon abatement %,carbon target,Vessel Speed,coalition Net present value,Firm Net present value,carbon_emissions_sim,sulfur_emissions_sim
0,0.01,3.395239e+08,"[16.67, 16.67, 16.67, 16.67, 16.67]",4.857956e+10,"[12474997794.581068, 11748680211.14705, 107426...","[[1698708.7371579632, 1749669.999272702, 17496...","[[36019.97725675727, 37100.57657445999, 37100...."
1,0.02,3.360944e+08,"[16.67, 16.67, 16.67, 16.67, 16.67]",4.857956e+10,"[12474997794.581068, 11748680211.14705, 107426...","[[1698708.7371579632, 1749669.999272702, 17496...","[[36019.97725675727, 37100.57657445999, 37100...."
2,0.03,3.326649e+08,"[16.67, 16.67, 16.67, 16.67, 16.67]",4.857956e+10,"[12474997794.581068, 11748680211.14705, 107426...","[[1698708.7371579632, 1749669.999272702, 17496...","[[36019.97725675727, 37100.57657445999, 37100...."
3,0.04,3.292353e+08,"[16.67, 16.67, 16.67, 16.67, 16.67]",4.857956e+10,"[12474997794.581068, 11748680211.14705, 107426...","[[1698708.7371579632, 1749669.999272702, 17496...","[[36019.97725675727, 37100.57657445999, 37100...."
4,0.05,3.258058e+08,"[16.67, 16.67, 16.67, 16.67, 16.67]",4.857956e+10,"[12474997794.581068, 11748680211.14705, 107426...","[[1698708.7371579632, 1749669.999272702, 17496...","[[36019.97725675727, 37100.57657445999, 37100...."
5,0.06,3.223763e+08,"[16.67, 16.67, 16.67, 16.67, 16.67]",4.857956e+10,"[12474997794.581068, 11748680211.14705, 107426...","[[1698708.7371579632, 1749669.999272702, 17496...","[[36019.97725675727, 37100.57657445999, 37100...."
6,0.07,3.189467e+08,"[16.67, 16.67, 16.67, 16.67, 16.67]",4.857956e+10,"[12474997794.581068, 11748680211.14705, 107426...","[[1698708.7371579632, 1749669.999272702, 17496...","[[36019.97725675727, 37100.57657445999, 37100...."
7,0.08,3.155172e+08,"[16.67, 16.67, 16.67, 16.67, 16.67]",4.857956e+10,"[12474997794.581068, 11748680211.14705, 107426...","[[1698708.7371579632, 1749669.999272702, 17496...","[[36019.97725675727, 37100.57657445999, 37100...."
8,0.09,3.120877e+08,"[16.67, 16.67, 16.67, 16.67, 16.67]",4.857956e+10,"[12474997794.581068, 11748680211.14705, 107426...","[[1698708.7371579632, 1749669.999272702, 17496...","[[36019.97725675727, 37100.57657445999, 37100...."
9,0.10,3.086581e+08,"[16.67, 16.67, 16.67, 16.67, 16.67]",4.857956e+10,"[12474997794.581068, 11748680211.14705, 107426...","[[1698708.7371579632, 1749669.999272702, 17496...","[[36019.97725675727, 37100.57657445999, 37100...."


In [24]:
coalition_results = pd.DataFrame(columns=["carbon abatement %", "carbon target" , "firm", "OptimumSpeed" ,"NPV_optimum", "carbon_optimum","sulfur_optimum" ])
coalition_results 




,carbon abatement %,carbon target,firm,OptimumSpeed,NPV_optimum,carbon_optimum,sulfur_optimum


# Output Analysis 

In [25]:
coalition_out = pd.DataFrame(columns=["carbon target","carbon abatement %","firm","OptimumSpeed","NPV_optimum","carbon_optimum","sulfur_optimum","Portion of carbon emissions %","Portion of sulfur emissions %","ITCM (revenue change)","ITCM (%)"])  
coalition_out

,carbon target,carbon abatement %,firm,OptimumSpeed,NPV_optimum,carbon_optimum,sulfur_optimum,Portion of carbon emissions %,Portion of sulfur emissions %,ITCM (revenue change),ITCM (%)


In [26]:
for my_in in (sum_short).index : 
    abatement_percentage_target = np.repeat(sum_short["carbon abatement %"][my_in],6)
    abatement_target = np.repeat(sum_short["carbon target"][my_in],6)
    my_firms =  [1,2,3,4,5, "coalition"]

    my_speed = sum_short["Vessel Speed"][my_in]
    speed_all = np.insert(my_speed, 5, nan)
    speed_all 
    my_NPV = sum_short["Firm Net present value"][my_in]
    NPV_coalition  = sum_short["coalition Net present value"][my_in]
    NPV_all = np.insert(my_NPV, 5, NPV_coalition)
    NPV_all
                
    carbon_1 = np.sum(sum_short["carbon_emissions_sim"][my_in][0])
    carbon_2 = np.sum(sum_short["carbon_emissions_sim"][my_in][1])
    carbon_3 = np.sum(sum_short["carbon_emissions_sim"][my_in][2])
    carbon_4 = np.sum(sum_short["carbon_emissions_sim"][my_in][3])
    carbon_5 = np.sum(sum_short["carbon_emissions_sim"][my_in][4])
    carbon_coal = carbon_1+carbon_2+carbon_3+carbon_4+carbon_5
    carbon_all = [carbon_1,carbon_2,carbon_3,carbon_4,carbon_5,carbon_coal]
                
    sulfur_1 = np.sum(sum_short["sulfur_emissions_sim"][my_in][0])
    sulfur_2 = np.sum(sum_short["sulfur_emissions_sim"][my_in][1])
    sulfur_3 = np.sum(sum_short["sulfur_emissions_sim"][my_in][2])
    sulfur_4 = np.sum(sum_short["sulfur_emissions_sim"][my_in][3])
    sulfur_5 = np.sum(sum_short["sulfur_emissions_sim"][my_in][4])
    sulfur_coal = sulfur_1+sulfur_2+sulfur_3+sulfur_4+sulfur_5
    sulfur_all = [sulfur_1,sulfur_2,sulfur_3,sulfur_4,sulfur_5,sulfur_coal]

        
    coalition_results = {
                        "carbon target" :abatement_target , 
                        "carbon abatement %" : abatement_percentage_target , 
                        "firm": my_firms, 
                        "OptimumSpeed": speed_all , 
        "NPV_optimum":NPV_all,
        
            "carbon_optimum"  :carbon_all,
        "sulfur_optimum":sulfur_all
                        }

    coalition_results_df = pd.DataFrame(data=coalition_results)


    # Adding the proportions of each firm from the coalition  
    coalition_results_df["Portion of carbon emissions %"] = 100*( coalition_results_df["carbon_optimum"]/ coalition_results_df["carbon_optimum"][5])
    coalition_results_df["Portion of sulfur emissions %"] = 100*( coalition_results_df["sulfur_optimum"]/ coalition_results_df["sulfur_optimum"][5])

     ## Incentive to change memership (loss in Revenus when joining the coaltion )
     ### Read in the BAU optimum solutions 

    speed_optim_1 = pd.read_csv('Speed_optimisation_1.csv') #import IMF data
    speed_optim_2 = pd.read_csv('Speed_optimisation_2.csv') #import IMF data
    speed_optim_3 = pd.read_csv('Speed_optimisation_3.csv') #import IMF data
    speed_optim_4 = pd.read_csv('Speed_optimisation_4.csv') #import IMF data
    speed_optim_5 = pd.read_csv('Speed_optimisation_5.csv') #import IMF data

    Industry_NPV  = speed_optim_1["NPV_optimum"][0] + speed_optim_2["NPV_optimum"][0] +speed_optim_3["NPV_optimum"][0]+ speed_optim_4["NPV_optimum"][0] +speed_optim_5["NPV_optimum"][0]
        
    NPV_BAU = [speed_optim_1["NPV_optimum"][0],speed_optim_2["NPV_optimum"][0], speed_optim_3["NPV_optimum"][0], 
             speed_optim_4["NPV_optimum"][0], speed_optim_5["NPV_optimum"][0], Industry_NPV ]


    coalition_results_df["ITCM (revenue change)"] =  coalition_results_df["NPV_optimum"]- NPV_BAU
    coalition_results_df["ITCM (%)"] =  ((coalition_results_df["NPV_optimum"]- NPV_BAU)/NPV_BAU)*100

    coalition_out = coalition_out.append(coalition_results_df)



In [27]:
speed_optim_1["NPV_optimum"][0]

12477054898.936825

In [29]:
coalition_out

,carbon target,carbon abatement %,firm,OptimumSpeed,NPV_optimum,carbon_optimum,sulfur_optimum,Portion of carbon emissions %,Portion of sulfur emissions %,ITCM (revenue change),ITCM (%)
0,3.395239e+08,0.01,1,16.67,1.247500e+10,6.164614e+07,1.307165e+06,20.220934,20.430021,-2.057104e+06,-0.016487
1,3.395239e+08,0.01,2,16.67,1.174868e+10,5.996457e+07,1.267762e+06,19.669350,19.814184,-2.206072e+06,-0.018774
2,3.395239e+08,0.01,3,16.67,1.074268e+10,5.799259e+07,1.232931e+06,19.022510,19.269802,-2.017509e+06,-0.018777
3,3.395239e+08,0.01,4,16.67,8.353690e+09,6.291806e+07,1.326055e+06,20.638146,20.725262,-2.277788e+06,-0.027259
4,3.395239e+08,0.01,5,16.67,5.259512e+09,6.234161e+07,1.264342e+06,20.449060,19.760731,-3.076141e+07,-0.581471
...,...,...,...,...,...,...,...,...,...,...,...
1,2.057721e+08,0.40,2,12.00,9.851063e+09,3.426888e+07,6.589387e+05,17.076589,16.755809,-1.899823e+09,-16.167489
2,2.057721e+08,0.40,3,12.00,8.361342e+09,3.286853e+07,6.406534e+05,16.378778,16.290842,-2.383359e+09,-22.181719
3,2.057721e+08,0.40,4,12.00,5.542796e+09,3.612212e+07,6.893471e+05,18.000082,17.529048,-2.813172e+09,-33.666620
4,2.057721e+08,0.40,5,16.67,5.259512e+09,6.234161e+07,1.264342e+06,31.065567,32.150299,-3.076141e+07,-0.581471


In [30]:
coalition_out.to_csv("coalition_results_sim_target_all_with_last_2.csv", index=False)